In [ ]:
1+1

In [ ]:
import uproot
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np 
import matplotlib as mpl

from util import *
import var
import cut
import data
import hist

import importlib

from pyanalib import panda_helpers

In [ ]:
importlib.reload(var)
importlib.reload(cut)
importlib.reload(data)
importlib.reload(hist)
importlib.reload(panda_helpers)

In [ ]:
plt.rcParams.update({'font.size': 16})
mpl.rcParams['lines.linewidth'] = 4.

dosave = False
savedir = "plots_9_19_23/reprodD_"

savedata = False
datadir = "/icarus/data/users/gputnam/dimuon-data/"

Masses = [220, 240, 260, 280] #300, 330]

In [ ]:
# fnu = "/icarus/data/users/gputnam/dimuon-data/mcnu_phase2_evt_new.df"
# fnu = "/icarus/data/users/gputnam/dimuon-data/nu-template-B/evt.df"
# fnu = "/icarus/data/users/gputnam/DMCP2023G/mc/mcnuphase2_evt_reprodD.df"
# fnu = "/icarus/data/users/gputnam/DMCP2023G/mc/mcnuphase2_evt_reprodD.df"
fnu = "/icarus/data/users/gputnam/DMCP2023G/mc/dfs-wstubs/mcnuphase2_evt_reprodD.df"

In [ ]:
nu_evtdf = pd.read_hdf(fnu, key="evt")

In [ ]:
nu_evtdf

In [ ]:
nu_mcdf = pd.read_hdf(fnu, key="mcnu")

In [ ]:
del nu_mcdf.wgt

In [ ]:
nu_mcdf["idx"] = nu_mcdf.groupby(level=[0,1]).cumcount()

In [ ]:
nu_mcdf.set_index(["idx"], append=True, inplace=True)

In [ ]:
nu_mcdf

In [ ]:
nu_hdr = pd.read_hdf(fnu, key="hdr")

In [ ]:
nu_hdr

In [ ]:
fhs = ["/icarus/data/users/gputnam/dimuon-data/higgs-gen-A/higgs_M%i_DMCP2023G_evt_new.df" % M for M in Masses]

fhs = ["/icarus/data/users/gputnam/dimuon-data/higgs-gen-D/higgs_M%i_DMC2023G_evt.df" % M for M in Masses]


In [ ]:
h_evtdfs = [pd.read_hdf(fh, key="evt") for fh in fhs]

In [ ]:
h_mcdfs = [pd.read_hdf(fh, key="mcnu") for fh in fhs]

In [ ]:
nlevel = 3
def pad(c):
    return tuple(list(c) + [""]*(nlevel - len(c)))

for i in range(len(h_mcdfs)):
    h_mcdfs[i].columns = pd.MultiIndex.from_tuples([pad(c) for c in h_mcdfs[i].columns])

In [ ]:
h_mchdfs = [pd.read_hdf(fh, key="mch") for fh in fhs]

In [ ]:
h_hdrs = [pd.read_hdf(fh, key="hdr") for fh in fhs]

In [ ]:
for i in range(len(h_mchdfs)):
    h_mchdfs[i].index = h_mchdfs[i].index.droplevel(-1)

In [ ]:
thetas = [h_mchdf.C1.iloc[0] for h_mchdf in h_mchdfs]
thetas

In [ ]:
from math import floor, log10

def fexp(f):
    return int(floor(log10(abs(f)))) if f != 0 else 0

def fman(f):
    return f/10**fexp(f)

theta_exps = [fexp(t) for t in thetas]
theta_mans = [fman(t) for t in thetas]

theta_strs = ["%.0f$\\times 10^{%i}$" % (m, e) for m,e in zip(theta_mans, theta_exps)]
theta_strs

In [ ]:
# MeVPrtl truth information

# Simulation parameters
for i in range(len(h_mchdfs)):
    p_M = h_mchdfs[i].M
    p_M.name = "p_M"

    p_th = h_mchdfs[i].C1
    p_th.name = "p_th"

    p_dist = dmagdf(h_mchdfs[i].start, h_mchdfs[i].enter)
    p_dist.name = "p_dist"

    p_raydist = dmagdf(h_mchdfs[i].enter, h_mchdfs[i].exit)
    p_raydist.name = "p_raydist"

    p_decaylength = h_mchdfs[i].decay_length
    p_decaylength.name = "p_decaylength"

    h_evtdfs[i][p_M.name] = p_M
    h_evtdfs[i][p_th.name] = p_th
    h_evtdfs[i][p_raydist.name] = p_raydist
    h_evtdfs[i][p_dist.name] = p_dist
    h_evtdfs[i][p_decaylength.name] = p_decaylength
    
    h_mcdfs[i][p_M.name] = p_M
    h_mcdfs[i][p_th.name] = p_th
    h_mcdfs[i][p_raydist.name] = p_raydist
    h_mcdfs[i][p_dist.name] = p_dist
    h_mcdfs[i][p_decaylength.name] = p_decaylength


nu_evtdf[p_M.name] = np.nan
nu_evtdf[p_th.name] = np.nan
nu_evtdf[p_dist.name] = np.nan
nu_evtdf[p_raydist.name] = np.nan
nu_evtdf[p_dist.name] = p_raydist
nu_evtdf[p_decaylength.name] = np.nan

nu_mcdf[p_M.name] = np.nan
nu_mcdf[p_th.name] = np.nan
nu_mcdf[p_dist.name] = np.nan
nu_mcdf[p_raydist.name] = np.nan
nu_mcdf[p_dist.name] = p_raydist
nu_mcdf[p_decaylength.name] = np.nan

In [ ]:
def sample_concat(dfA, dfB, offset=1_000_000):
    dfB = dfB.copy()
    # dfB.index = dfB.index.set_levels(dfB.index.levels[0] + dfA.index.max()[0] + 1, level=0)
    dfB.index = dfB.index.set_levels(dfB.index.levels[0] + offset, level=0)
    return dfA.append(dfB)

In [ ]:
nu_pot = np.sum(nu_hdr.pot * nu_hdr.first_in_subrun)
h_pots = [np.sum(h_hdr.pot * h_hdr.first_in_subrun) for h_hdr in h_hdrs]

In [ ]:
nu_evt_per_spill = np.sum(nu_hdr.ngenevt*nu_hdr.first_in_subrun) / (np.sum(nu_hdr.pot*nu_hdr.first_in_subrun) / 6e13)
# ic_pot = np.sum(ic_hdr.ngenevt * ic_hdr.first_in_subrun * 6e13) / (1 - nu_evt_per_spill)

In [ ]:
GOAL_POT = 2e20 
nu_evtdf["scale"] = GOAL_POT / nu_pot
# ic_evtdf["scale"] = GOAL_POT / ic_pot
nu_mcdf["scale"] = GOAL_POT / nu_pot

for i in range(len(h_mchdfs)):
    h_evtdfs[i]["scale"] = GOAL_POT / h_pots[i]
    h_mcdfs[i]["scale"] = GOAL_POT / h_pots[i]

POTSTR = "2e20 POT"

In [ ]:
for i in range(len(h_mchdfs)):
    h_evtdfs[i]["higgs"] = True
    
nu_evtdf["higgs"] = False
# ic_evtdf["higgs"] = False

In [ ]:
def decay_weight(in_dist, out_dist, mean_dist):
    return np.exp(-in_dist / mean_dist) - \
          np.exp(-out_dist / mean_dist)
    
def flux_weight(mixing):
    return mixing * mixing

def reweight_mixing(newmixing, start, enter, exit, mean_dist, oldmixing=1e-5):
    dist_in = dist(enter, start)
    dist_out = dist(exit, start)
    
    old_dcy = decay_weight(dist_in, dist_out, mean_dist)
    new_dcy = decay_weight(dist_in, dist_out, mean_dist / (newmixing**2 / oldmixing**2))
    
    old_flux = flux_weight(oldmixing)
    new_flux = flux_weight(newmixing)
    
    return (new_flux / old_flux) * (new_dcy / old_dcy)

In [ ]:
h_evtdf = h_evtdfs[0].copy()
for i in range(1, len(h_evtdfs)):
    h_evtdf = sample_concat(h_evtdf, h_evtdfs[i], offset=1_000_000*i)

In [ ]:
h_mcdf = h_mcdfs[0].copy()
for i in range(1, len(h_evtdfs)):
    h_mcdf = sample_concat(h_mcdf, h_mcdfs[i], offset=1_000_000*i)

In [ ]:
evtdf = sample_concat(h_evtdf, nu_evtdf, offset=1_000_000*len(h_evtdfs))

In [ ]:
is_higgs = evtdf.higgs & (evtdf.slc.truth.npi == 0) & (evtdf.slc.truth.npi0 == 0)
is_higgs.name = "Scalar"

is_higgs_minM = is_higgs & (evtdf.p_M == Masses[0]/1e3)
is_higgs_minM.name = "Scalar"

is_nu = (evtdf.slc.tmatch.idx >= 0) & ~evtdf.higgs
is_nu.name = "$\\nu$"
is_cosmic = (evtdf.slc.tmatch.idx < 0) & ~evtdf.higgs
is_cosmic.name = "Cosmic"

In [ ]:
categories = [is_higgs, is_nu, is_cosmic]

In [ ]:
nucategories = [
    (np.abs(evtdf.slc.truth.pdg) == 14) & (evtdf.slc.truth.iscc==1) & ((evtdf.slc.truth.genie_mode == 0) | (evtdf.slc.truth.genie_mode == 10)), # CC QE+MEC
    (np.abs(evtdf.slc.truth.pdg) == 14) & (evtdf.slc.truth.iscc==1) & (evtdf.slc.truth.genie_mode == 1), # CC RES
    (np.abs(evtdf.slc.truth.pdg) == 14) & (evtdf.slc.truth.iscc==1) & (evtdf.slc.truth.genie_mode == 2), # CC DIS
    (np.abs(evtdf.slc.truth.pdg) == 14) & (evtdf.slc.truth.iscc==1) & (evtdf.slc.truth.genie_mode == 3), # CC COH
    (evtdf.slc.truth.iscc == 0)# NC
]

nucategories = [nu & ~evtdf.higgs for nu in nucategories]

nucategories[0].name = "$\\nu_\\mu$ CC QE+MEC"
nucategories[1].name = "$\\nu_\\mu$ CC RES"
nucategories[2].name = "$\\nu_\\mu$ CC DIS"
nucategories[3].name = "$\\nu_\\mu$ CC COH"
nucategories[4].name = "$\\nu$ NC"

In [ ]:
LAr_density_gmL = 1.389875

# RECOMBINATION

# ArgoNeuT params
MODA = 0.930
MODB = 0.212
Wion = 1e3 / 4.237e7

Vps = 75058
Enom = (375./385)*Vps*1e-3 /148.25
Eshort = (368.6/378.6)*Vps*1e-3 /148.25
Efield = Enom

def recombination(dEdx, A=MODA, B=MODB, E=Efield):
    alpha = A
    beta = B / (LAr_density_gmL * E)

    dQdx = np.log(alpha + dEdx*beta) / (Wion * beta)
    return dQdx

# recombination correction factor, MC
Rfactor = 1/(recombination(1.6, A=0.93, B=0.212, E=0.494)*Wion)
Gain = 1/0.01265
Rfactor

In [ ]:
beam2det = np.array([ [0.921035925, 0.022715103, 0.388814672], [0, 0.998297825, -0.058321970], [-0.389477631, 0.053716629, 0.919468161]])

beamorigin = np.array([4.503730e2, 80.153901e2, 795.112945e2])

MUON_MASS = 0.1057 
PION_MASS = 0.13457

BEAMDIR = beam2det.dot(beamorigin) / np.linalg.norm(beam2det.dot(beamorigin)) 
BEAMDIR

In [ ]:
def recop(trk):
    p = trk.rangeP.p_muon
    p[~TrkInFV(trk.end)] = trk.mcsP.fwdP_muon
    return p

def is_muon(trk):
    return (TrkInFV(trk.end) & (trk.chi2pid.I2.chi2_muon < 40) & (trk.chi2pid.I2.chi2_proton > 80)) | (~TrkInFV(trk.end) & (trk.len > 100.))

def is_proton(trk):
    return TrkInFV(trk.end) & (trk.chi2pid.I2.chi2_muon > 40) & (trk.chi2pid.I2.chi2_proton < 60)

In [ ]:
scalar_mom = evtdf.trunk.trk.dir.multiply(recop(evtdf.trunk.trk), axis=0) + \
           evtdf.branch.trk.dir.multiply(recop(evtdf.branch.trk), axis=0)
    
scalar_dir = scalar_mom.divide(magdf(scalar_mom), axis=0)

beamangle = np.arccos(scalar_dir.x*BEAMDIR[0] + scalar_dir.y*BEAMDIR[1] + scalar_dir.z*BEAMDIR[2])

trunk_beamcos = evtdf.trunk.trk.dir.x*BEAMDIR[0] + evtdf.trunk.trk.dir.y*BEAMDIR[1] + evtdf.trunk.trk.dir.z*BEAMDIR[2]
trunk_E = np.sqrt(recop(evtdf.trunk.trk)**2 + MUON_MASS**2)
branch_beamcos = evtdf.branch.trk.dir.x*BEAMDIR[0] + evtdf.branch.trk.dir.y*BEAMDIR[1] + evtdf.branch.trk.dir.z*BEAMDIR[2]
branch_E = np.sqrt(recop(evtdf.branch.trk)**2 + PION_MASS**2)


coh_t = (np.sin(beamangle)*magdf(scalar_mom))**2 + (trunk_E - recop(evtdf.trunk.trk)*trunk_beamcos + branch_E - recop(evtdf.branch.trk)*branch_beamcos)**2

In [ ]:

Etrunk = np.sqrt(MUON_MASS**2 + recop(evtdf.trunk.trk)**2)
Ebranch = np.sqrt(MUON_MASS**2 + recop(evtdf.branch.trk)**2)

s_mass = np.sqrt(2)*np.sqrt(MUON_MASS**2 + Etrunk*Ebranch - recop(evtdf.trunk.trk)*recop(evtdf.branch.trk)*dotdf(evtdf.branch.trk.dir, evtdf.trunk.trk.dir))

In [ ]:
trunk_mcs_range = np.abs(1 - evtdf.trunk.trk.mcsP.fwdP_muon / evtdf.trunk.trk.rangeP.p_muon)
branch_mcs_range = np.abs(1 - evtdf.branch.trk.mcsP.fwdP_muon / evtdf.branch.trk.rangeP.p_muon)

In [ ]:
# OPTIMIZED
MAX_SHW_LEN_CUT = 15
MAX_OTHR_TRK_LEN_CUT = 20
OTHR_CHI2P_CUT = 80
OTHR_CHI2U_CUT = 40

LEN_CUT = 25
TRK_SCORE_CUT = 0.1
CHI2U_CUT = 10
CHI2P_CUT = 80
MED_DQDX_CUT = 0
MCS_RANGE_COMP = 0.5

STUB_5mm_CUT = 5e5 / 1e3
STUB_10mm_CUT = 3e5 / 1e3
STUB_20mm_CUT = 2.5e5 / 1e3
STUB_30mm_CUT = 2e5 / 1e3

OPENANGLE_CUT = 0.8
UU_ANGLE_CUT = 0.125

In [ ]:
# # LOOSE
# MAX_SHW_LEN_CUT = 15
# OTHR_CHI2P_CUT = 80
# OTHR_CHI2U_CUT = 40

# LEN_CUT = 25
# TRK_SCORE_CUT = 0
# CHI2U_CUT = 20
# CHI2P_CUT = 80
# MED_DQDX_CUT = 0

# STUB_5mm_CUT = 5e5 / 1e3
# STUB_10mm_CUT = 3e5 / 1e3
# STUB_20mm_CUT = 2.5e5 / 1e3
# STUB_30mm_CUT = 2e5 / 1e3

# OPENANGLE_CUT = 0.6
# UU_ANGLE_CUT = 0.15

In [ ]:
preselect = SlcInFV(evtdf.slc.vertex) & TrkInFV(evtdf.trunk.trk.end) & TrkInFV(evtdf.branch.trk.end)

In [ ]:
var = evtdf.max_othr_trk_len
when = preselect
bins = np.linspace(0, 20, 41)
i_fig = 0

for M in Masses:
    plt.figure(i_fig)
    i_fig += 1

    this_categories = [is_higgs, is_nu, is_cosmic]
    this_categories[0] = is_higgs & (evtdf.p_M == M/1e3)
    this_categories[0].name = "Scalar"
    for c in this_categories:
        _ = plt.hist(var[when & c], weights=evtdf.scale[when & c], bins=bins, label=c.name, 
                     histtype="step", linewidth=2, density=True)
    plt.legend()
#     plt.axvline([cutvals[i_cut]], linestyle=":", color="r", linewidth=2)

    plt.tight_layout()
#     plt.xlabel(cutnames[i_cut])
    plt.title("ICARUS MC, $M_S = %i$ MeV"% M)
    plt.ylabel("Area Normalized")

#     plt.text(0.4, 0.05, label, transform=plt.gca().transAxes, fontsize=18)

    plt.tight_layout()
    if dosave: 
        plt.savefig(savedir + "%s_M%i.pdf" % (cutsnames[i_cut], M))
        plt.savefig(savedir + "%s_M%i.svg" % (cutsnames[i_cut], M))

In [ ]:
var = evtdf.max_othr_trk_len_primary
when = preselect
bins = np.linspace(0, 20, 41)
i_fig = 0


for M in Masses:
    plt.figure(i_fig)
    i_fig += 1

    this_categories = [is_higgs, is_nu, is_cosmic]
    this_categories[0] = is_higgs & (evtdf.p_M == M/1e3)
    this_categories[0].name = "Scalar"
    for c in this_categories:
        _ = plt.hist(var[when & c], weights=evtdf.scale[when & c], bins=bins, label=c.name, 
                     histtype="step", linewidth=2, density=True)
    plt.legend()
#     plt.axvline([cutvals[i_cut]], linestyle=":", color="r", linewidth=2)

    plt.tight_layout()
#     plt.xlabel(cutnames[i_cut])
    plt.title("ICARUS MC, $M_S = %i$ MeV"% M)
    plt.ylabel("Area Normalized")

#     plt.text(0.4, 0.05, label, transform=plt.gca().transAxes, fontsize=18)

    plt.tight_layout()
    if dosave: 
        plt.savefig(savedir + "%s_M%i.pdf" % (cutsnames[i_cut], M))
        plt.savefig(savedir + "%s_M%i.svg" % (cutsnames[i_cut], M))

In [ ]:
evtdf[["min_othr_chi2_proton", "min_othr_chi2_proton_all"]]

In [ ]:
cutvars = [
    evtdf.max_shw_len_primary,
    evtdf.max_othr_trk_len,
    evtdf.max_othr_chi2_muon,
    evtdf.min_othr_chi2_proton_all,
    evtdf.trunk.trk.len,
    evtdf.branch.trk.len,
    evtdf.trunk.trackScore,
    evtdf.branch.trackScore,
    evtdf.trunk.trk.chi2pid.I2.chi2_muon,
    evtdf.trunk.trk.chi2pid.I2.chi2_proton,
    evtdf.branch.trk.chi2pid.I2.chi2_muon,
    evtdf.branch.trk.chi2pid.I2.chi2_proton,
    trunk_mcs_range,
    branch_mcs_range,
    evtdf.stub.l0_5cm.dqdx/1e3,
    evtdf.stub.l1cm.dqdx/1e3,
    evtdf.stub.l2cm.dqdx/1e3,
    evtdf.stub.l3cm.dqdx/1e3,
    np.arccos(dotdf(evtdf.trunk.trk.dir, evtdf.branch.trk.dir)),
    beamangle,    
]

evtdf["Truthy"] = True

wherecut = [
    ~np.isnan(evtdf.max_shw_len),
    ~np.isnan(evtdf.max_othr_trk_len),
    ~np.isnan(evtdf.max_othr_chi2_muon),
    ~np.isnan(evtdf.min_othr_chi2_proton_all),
    evtdf.Truthy,
    evtdf.Truthy,
    evtdf.Truthy,
    evtdf.Truthy,
    evtdf.Truthy,
    evtdf.Truthy,
    evtdf.Truthy,
    evtdf.Truthy,
    evtdf.Truthy,
    evtdf.Truthy,
    ~np.isnan(evtdf.stub.l0_5cm.dqdx),
    ~np.isnan(evtdf.stub.l1cm.dqdx),
    ~np.isnan(evtdf.stub.l2cm.dqdx),
    ~np.isnan(evtdf.stub.l3cm.dqdx),
    evtdf.Truthy,
    evtdf.Truthy,
]

cut_hi = [
    True,
    True,
    True,
    False,
    False,
    False,
    False,
    False,
    True,
    False,
    True,
    False,
    True,
    True,
    True,
    True,
    True,
    True,
    True, #False,
    True,
]

cutvals = [
    MAX_SHW_LEN_CUT,
    MAX_OTHR_TRK_LEN_CUT,
    OTHR_CHI2U_CUT,
    OTHR_CHI2P_CUT,
    LEN_CUT,
    LEN_CUT,
    TRK_SCORE_CUT,
    TRK_SCORE_CUT,
    CHI2U_CUT,
    CHI2P_CUT,
    CHI2U_CUT,
    CHI2P_CUT,
    MCS_RANGE_COMP,
    MCS_RANGE_COMP,
    STUB_5mm_CUT,
    STUB_10mm_CUT,
    STUB_20mm_CUT,
    STUB_30mm_CUT,
    OPENANGLE_CUT,
    UU_ANGLE_CUT,
]

N_TEST_CUT = 41
cutranges = [
    np.linspace(0, 20, N_TEST_CUT), # MAX SHW LEN
    np.linspace(0, 60, N_TEST_CUT), # MAX OTHR TRK LEN
    np.linspace(0, 100, N_TEST_CUT), # OTHR CHI2U
    np.linspace(0, 200, N_TEST_CUT), # OTHR CHI2P
    np.linspace(0, 40, N_TEST_CUT), # TRUNK LEN
    np.linspace(0, 40, N_TEST_CUT), # BRANCH LEN
    np.linspace(0, 1, N_TEST_CUT), # TRUNK TRACK SCORE
    np.linspace(0, 1, N_TEST_CUT), # BRANCH TRACK SCORE
    np.linspace(5, 20, N_TEST_CUT), # TRUNK CHI2U
    np.linspace(80, 200, N_TEST_CUT), # TRUNK CHI2P
    np.linspace(5, 20, N_TEST_CUT), # BRANCH CHI2U
    np.linspace(80, 200, N_TEST_CUT), # BRANCH CHI2P
    np.linspace(0.25, 1, N_TEST_CUT), # MCS RANGE
    np.linspace(0.25, 1, N_TEST_CUT), # MCS RANGE
    np.linspace(0, 8e5, N_TEST_CUT) / 1e3, # STUB 5mm
    np.linspace(0, 8e5, N_TEST_CUT) / 1e3, # STUB 10mm
    np.linspace(0, 8e5, N_TEST_CUT) / 1e3, # STUB 20mm
    np.linspace(0, 8e5, N_TEST_CUT) / 1e3, # STUB 30mm
    np.linspace(0.4, 1, N_TEST_CUT), # OPEN ANGLE
    np.linspace(0.05, 0.5, N_TEST_CUT), # UU ANGLE
]

cuts = [cutvars[i_cut] < cutvals[i_cut] if cut_hi[i_cut] else cutvars[i_cut] > cutvals[i_cut] 
        for i_cut in range(len(cutvals))]
cuts = [cuts[i_cut] | ~wherecut[i_cut] for i_cut in range(len(cuts))]

cutnames =[
    "Max Shower Length [cm]",
    "Max Other Track Length [cm]",
    "Max Other $\\chi^2_\\mu$",
    "Min Other $\\chi^2_p$",
    "Trunk Length [cm]",
    "Branch Length [cm]",
    "Trunk Track Score",
    "Branch Track Score",
    "Trunk $\\chi^2_\\mu$",
    "Trunk $\\chi^2_p$",
    "Branch $\\chi^2_\\mu$",
    "Branch $\\chi^2_p$",
    "Trunk (1 - MCS P / Range P)",
    "Branch (1 - MCS P / Range P)",
    "Stub dQ/dx [ke$^-$/cm] (len < 0.5 cm)",
    "Stub dQ/dx [ke$^-$/cm] (0.5 cm < len < 1 cm)",
    "Stub dQ/dx [ke$^-$/cm] (1 cm < len < 2 cm)",
    "Stub dQ/dx [ke$^-$/cm] (2 cm < len < 3 cm)",
    "$\\mu\\mu$ Opening Angle",
    "$\\mu\\mu$ Angle to Beam",
]

cutsnames = [
    "max_shw_len",
    "max_othr_trk_len",
    "max_othr_chi2u",
    "min_othr_chi2p",
    "trunk_len",
    "branch_len",
    "trunk_trkscore",
    "branch_trkscore",
    "trunk_chi2u",
    "trunk_chi2p",
    "branch_chi2u",
    "branch_chi2p",
    "trunk_p_comp",
    "branch_p_comp",
    "stub_dqdx_5mm",
    "stub_dqdx_10mm",
    "stub_dqdx_20mm",
    "stub_dqdx_30mm",
    "uu_openangle",
    "uu_beamangle"
    
]

assert(len(cutvars) == len(wherecut) == len(cut_hi) == len(cutvals) == len(cutranges) == len(cuts) == len(cutnames))


In [ ]:
NCUT_Object = 4
NCUT_MuID = 10
NCUT_StubID = 4
NCUT_KINEMATICS = 2

cut_types = [
    [evtdf.Truthy],
    cuts[:NCUT_Object],
    cuts[NCUT_Object:NCUT_Object+NCUT_MuID],
    cuts[NCUT_Object+NCUT_MuID:NCUT_Object+NCUT_MuID+NCUT_StubID],
    cuts[NCUT_Object+NCUT_MuID+NCUT_StubID:]
]

cut_type_names = [
    "Pre-Selection",
    "Object",
    "Muon ID",
    "Stub ID",
    "Kinematic"
]

In [ ]:
def cut_all(l):
    l0 = l[0]
    for c in l[1:]:
        l0 = l0 & c
    return l0

def cut_any(l):
    l0 = l[0]
    for c in l[1:]:
        l0 = l0 | c
    return l0

In [ ]:
preselect = SlcInFV(evtdf.slc.vertex) & TrkInFV(evtdf.trunk.trk.end) & TrkInFV(evtdf.branch.trk.end)

In [ ]:
i_fig = 0
for i_cut in range(len(cutvars)):
    var = cutvars[i_cut]
    when = preselect
    bins = cutranges[i_cut]
    
    for M in Masses:
        plt.figure(i_fig)
        i_fig += 1
        
        this_categories = [is_higgs, is_nu, is_cosmic]
        this_categories[0] = is_higgs & (evtdf.p_M == M/1e3)
        this_categories[0].name = "Scalar"
        for c in this_categories:
            _ = plt.hist(var[when & c], weights=evtdf.scale[when & c], bins=bins, label=c.name, 
                         histtype="step", linewidth=2, density=True)
        plt.legend()
        plt.axvline([cutvals[i_cut]], linestyle=":", color="r", linewidth=2)

        plt.tight_layout()
        plt.xlabel(cutnames[i_cut])
        plt.title("ICARUS MC, $M_S = %i$ MeV"% M)
        plt.ylabel("Area Normalized")

    #     plt.text(0.4, 0.05, label, transform=plt.gca().transAxes, fontsize=18)

        plt.tight_layout()
        if dosave: 
            plt.savefig(savedir + "%s_M%i.pdf" % (cutsnames[i_cut], M))
            plt.savefig(savedir + "%s_M%i.svg" % (cutsnames[i_cut], M))

In [ ]:
for i_cut in range(len(cutvars)):
    plt.figure(i_cut)
    cut = cut_all([cuts[j_cut] for j_cut in range(len(cuts)) if j_cut != i_cut])
    # Per mass
    for M in Masses:
        when = preselect & cut & (np.abs(s_mass - M/1e3) < 0.02)
        signal = evtdf.higgs & (evtdf.p_M == M/1e3) & when
        bkg = ~evtdf.higgs & when
        
        vary_cut = [cutvars[i_cut] < C if cut_hi[i_cut] else cutvars[i_cut] > C for C in cutranges[i_cut]]
        vary_cut = [c | ~wherecut[i_cut] for c in vary_cut]
        
        sig_counts = np.array([evtdf.scale[signal & c].sum() for c in vary_cut])
        bkg_counts = np.array([evtdf.scale[bkg & c].sum() for c in vary_cut])

        significance = sig_counts / np.sqrt(sig_counts + bkg_counts)
        tot_count = sig_counts + bkg_counts
        
        plt.plot(cutranges[i_cut][tot_count > 0], significance[tot_count > 0] / np.max(significance[tot_count > 0]), 
                    label="%i" % M, marker=".", markersize=15)
        
        # plt.axvline([cutvals[i_cut]], linestyle=":", color="red")
        
    plt.xlabel(cutnames[i_cut])
    plt.ylabel("Relative Significance (S/$\\sqrt{S+B}$)")

    plt.legend(title="Scalar Mass [MeV]", bbox_to_anchor=(0.5, 1.35), ncol=3, loc="upper center")
    if dosave: 
        plt.savefig(savedir + "%s_sig.pdf" % (cutsnames[i_cut],), bbox_inches="tight") 
        plt.savefig(savedir + "%s_sig.svg" % (cutsnames[i_cut],), bbox_inches="tight") 

In [ ]:
bins = np.linspace(0, 5, 11)

for ifig,M in enumerate(Masses):
    plt.figure(ifig, figsize=(9, 6))
    
    when_all = SlcInFV(h_mcdf.position) & (h_mcdf.p_M == M/1e3) & (h_mcdf.npi == 0) & (h_mcdf.npi0 == 0) & TrkInFV(h_mcdf.p0.end) & TrkInFV(h_mcdf.p1.end)

    slc_is_reco = SlcInFV(evtdf.slc.vertex) & (evtdf.slc.tmatch.idx == 0) & TrkInFV(evtdf.trunk.trk.end) & TrkInFV(evtdf.branch.trk.end)
    # slc_is_select = slc_is_reco & passcut

    thiscut = True
    for c, name in zip(cut_types, cut_type_names):
        thiscut = cut_all(c) & thiscut
        when_cut = when_all & (slc_is_reco & thiscut)[is_higgs].groupby(level=[0, 1]).any()
        D, _ = np.histogram(h_mcdf[when_all].E, bins=bins)
        N, _ = np.histogram(h_mcdf[when_cut].E, bins=bins)  

        centers = (bins[1:] + bins[:-1])/2.

        plt.plot(centers, N/D, label=name)
    plt.legend(bbox_to_anchor=(1.04,1), loc="upper left", title="Cut Stage")

    plt.xlabel("Scalar Energy [GeV]")
    plt.ylabel("Efficiency")
    plt.title("ICARUS MC, $M_S = %i$ MeV"% M)

    plt.tight_layout()
    if dosave: 
        plt.savefig(savedir + "scalar_eff_all_M%i.pdf" % M)
        plt.savefig(savedir + "scalar_eff_all_M%i.svg" % M)

In [ ]:
var = s_mass
when = preselect & cut_all(cuts)
bins = np.linspace(0.2, 0.6, 9)

# assert((when & ~cut_any(nucategories) & ~evtdf.higgs).sum() == 0)

for i,M in enumerate(Masses):
    plt.figure(i)
    
    _ = plt.hist([var[when & nu] for nu in nucategories], 
                 weights=[evtdf.scale[when & nu] for nu in nucategories],
                bins=bins, stacked=True, label=[nu.name for nu in nucategories])
    
    this_is_higgs = is_higgs & (evtdf.p_M == M/1e3)
    _ = plt.hist(var[when & this_is_higgs], weights=evtdf.scale[when & this_is_higgs], bins=bins,
                 label="Scalar", histtype="step", edgecolor="black", linewidth=2)

    plt.legend(ncol=2)
    plt.tight_layout()
    plt.xlabel("$M_{\\mu\\mu}$ [GeV]")
    plt.title("ICARUS MC, $M_S = %i$ MeV, $\\theta_S = $%s"% (M, theta_strs[i]))
    plt.ylabel("Events / %s" % POTSTR)
    plt.ylim([0, plt.ylim()[1]*1.5])

    if dosave: 
        plt.savefig(savedir + "smass_M%i.pdf" % (M, ), bbox_inches="tight")
        plt.savefig(savedir + "smass_M%i.svg" % (M, ), bbox_inches="tight")
    

In [ ]:
var = beamangle*180/np.pi
when = preselect & cut_all(cuts[:-1])
bins = np.linspace(0., 75, 11)

# assert((when & ~cut_any(nucategories) & ~evtdf.higgs).sum() == 0)

for i,M in enumerate(Masses):
    plt.figure(i)
    
    _ = plt.hist([var[when & nu] for nu in nucategories], 
                 weights=[evtdf.scale[when & nu] for nu in nucategories],
                bins=bins, stacked=True, label=[nu.name for nu in nucategories])
    
    this_is_higgs = is_higgs & (evtdf.p_M == M/1e3)
    _ = plt.hist(var[when & this_is_higgs], weights=evtdf.scale[when & this_is_higgs], bins=bins,
                 label="Scalar", histtype="step", edgecolor="black", linewidth=2)

    plt.legend(ncol=2)
    plt.tight_layout()
    plt.xlabel("$\\theta_\\mathrm{NuMI}$ [$^\\circ$]")
    plt.title("ICARUS MC, $M_S = %i$ MeV, $\\theta_S = $%s"% (M, theta_strs[i]))
    plt.ylabel("Events / %s" % POTSTR)
    plt.ylim([0, plt.ylim()[1]*1.5])

    if dosave: 
        plt.savefig(savedir + "beamangle_M%i.pdf" % (M, ), bbox_inches="tight")
        plt.savefig(savedir + "beamangle_M%i.svg" % (M, ), bbox_inches="tight")
    

In [ ]:
width = 0.25
plt.figure(figsize=(6.4, 6.4))

m_ok = ~is_higgs | (evtdf.p_M == 0.26)

for i,category in enumerate(reversed(categories)):
    plt.barh(np.arange(len(cut_types)) + i*width, 
             [np.sum(evtdf.scale[m_ok & category & preselect & cut_all([c for cuts in cut_types[:i+1] for c in cuts])]) for i in range(len(cut_types))], 
             width, 
             label=category.name,
             tick_label = cut_type_names
            )

plt.xscale("log")
plt.legend(ncol=3, bbox_to_anchor=(-0.05, 0.92, 1, 0.2), loc="upper left")
plt.xlabel("Scalar Candidates / %s (1yr)" % POTSTR)
    
plt.xlim([1, plt.xlim()[1]])
# plt.title(label.replace("\n", ", "))
plt.tight_layout()

if dosave: 
    plt.savefig(savedir + "cutchart_M%i.svg" % (260))
    plt.savefig(savedir + "cutchart_M%i.pdf" % (260))

In [ ]:
sum(evtdf.scale[when & ~evtdf.higgs])

In [ ]:
for nu in nucategories:
    print(nu.name, evtdf.scale[(when & nu)].sum())

In [ ]:
evtdf.columns

In [ ]:
6.2/8.2

In [ ]:
cols = [
    "min_othr_chi2_proton",
    "max_othr_chi2_muon",
    "min_othr_chi2_proton_primary",
    "max_othr_chi2_muon_primary",
    "min_othr_chi2_proton_all",
    "max_othr_chi2_muon_all"
]

In [ ]:
evtdf.loc[when & nu, cols]

In [ ]:
for nu in nucategories:
    print(nu.name, evtdf.scale[(when & nu)].sum())

In [ ]:
var = evtdf.slc.truth.max_proton_ke

when_othrobj = preselect & (~evtdf.higgs) & cut_all(cuts[:4] + cuts[19:20])
when_caloPID = preselect & (~evtdf.higgs) & cut_all(cuts[:13] + cuts[19:20])
when_topoPID = preselect & (~evtdf.higgs) & cut_all(cuts[:15] + cuts[19:20])
when_stubPID = preselect & (~evtdf.higgs) & cut_all(cuts[:19] + cuts[19:20])
when_kinematics = preselect & (~evtdf.higgs) & cut_all(cuts)

bins = np.linspace(0, 2, 9)

_ = plt.hist(var[when_othrobj], weights=evtdf.scale[when_othrobj], bins=bins, histtype="step", linewidth=2, label="Before Particle ID")
_ = plt.hist(var[when_caloPID], weights=evtdf.scale[when_caloPID], bins=bins, histtype="step", linewidth=2, label="Calorimetric PID")
_ = plt.hist(var[when_topoPID], weights=evtdf.scale[when_topoPID], bins=bins, histtype="step", linewidth=2, label="Topological PID")
_ = plt.hist(var[when_stubPID], weights=evtdf.scale[when_stubPID], bins=bins, histtype="step", linewidth=2, label="Stub Low-E ID")
_ = plt.hist(var[when_kinematics], weights=evtdf.scale[when_kinematics], bins=bins, histtype="step", linewidth=2, label="Kinematics Cuts")
plt.yscale("log")
plt.ylim([1, plt.ylim()[1]])
plt.legend()

plt.ylabel("Selected $\\nu$ / %s" % POTSTR)
plt.xlabel("Maximum Proton Kinetic Energy [GeV]")

if dosave: 
    plt.savefig(savedir + "proton_rejection.pdf", bbox_inches="tight")
    plt.savefig(savedir + "proton_rejection.svg", bbox_inches="tight")

In [ ]:
evtdf[when & ~evtdf.higgs & (np.abs(evtdf.branch.trk.truth.p.pdg) == 2212)].branch.trk

In [ ]:
openangle = np.arccos(dotdf(evtdf.trunk.trk.dir, evtdf.branch.trk.dir))

In [ ]:
evtdf[when & ~evtdf.higgs & (np.abs(evtdf.branch.trk.truth.p.pdg) == 2212)].branch.trk.truth

In [ ]:
evtdf[when & ~evtdf.higgs & (np.abs(evtdf.trunk.trk.truth.p.pdg) == 2212)].trunk.trk.truth

In [ ]:
evtdf.slc.truth[when & ~evtdf.higgs & ((np.abs(evtdf.trunk.trk.truth.p.pdg) == 2212) | (np.abs(evtdf.branch.trk.truth.p.pdg) == 2212))][evtdf.slc.truth.columns[:20]]

In [ ]:
evtdf.slc.truth.columns[:20]

In [ ]:
trunk_mcs_range[when & ~evtdf.higgs]

In [ ]:
when_all = SlcInFV(nu_mcdf.position) #& TrkInFV(nu_mcdf.p0.end) & TrkInFV(nu_mcdf.p1.end)

In [ ]:
slc_is_reco = SlcInFV(evtdf.slc.vertex) & (evtdf.slc.tmatch.idx >= 0) & ~evtdf.higgs &\
    preselect & cut_all(cuts[:-1] + [beamangle < 0.4])

idx_reco = evtdf.slc.tmatch.idx[slc_is_reco]

mcnu_idx_reco = pd.MultiIndex.from_arrays(
    [idx_reco.index.get_level_values(0) - len(fhs)*1_000_000, idx_reco.index.get_level_values(1), idx_reco.values], 
    names=list(idx_reco.index.names[:2]) + ["idx"])

nu_mcdf["reco"] = False
nu_mcdf.loc[mcnu_idx_reco, "reco"] = True

when_cut = when_all & nu_mcdf.reco

In [ ]:
var = nu_mcdf.max_proton_ke
bins= np.linspace(0, 1, 21)

_ = plt.hist(var[when_all], weights=nu_mcdf.scale[when_all], bins=bins, histtype="step")
_ = plt.hist(var[when_cut], weights=nu_mcdf.scale[when_cut], bins=bins, histtype="step")
plt.yscale("log")

In [ ]:
var = nu_mcdf.npi + nu_mcdf.iscc
bins= np.linspace(0, 10, 11)

_ = plt.hist(var[when_all], weights=nu_mcdf.scale[when_all], bins=bins, histtype="step")
_ = plt.hist(var[when_cut], weights=nu_mcdf.scale[when_cut], bins=bins, histtype="step")
plt.yscale("log")

In [ ]:
var = nu_mcdf.genie_mode
bins = np.linspace(0, 10, 11)

_ = plt.hist(var[when_all], weights=nu_mcdf.scale[when_all], bins=bins, histtype="step")
_ = plt.hist(var[when_cut], weights=nu_mcdf.scale[when_cut], bins=bins, histtype="step")
plt.yscale("log")

In [ ]:
var = nu_mcdf.iscc
bins= np.linspace(0, 2, 3)

_ = plt.hist(var[when_all], weights=nu_mcdf.scale[when_all], bins=bins, histtype="step")
_ = plt.hist(var[when_cut], weights=nu_mcdf.scale[when_cut], bins=bins, histtype="step")
plt.yscale("log")

In [ ]:
var = InFV(nu_mcdf.position, 50, 10, 10, 10)
bins= np.linspace(0, 2, 3)

_ = plt.hist(var[when_all], weights=nu_mcdf.scale[when_all], bins=bins, histtype="step")
_ = plt.hist(var[when_cut], weights=nu_mcdf.scale[when_cut], bins=bins, histtype="step")
plt.yscale("log")

In [ ]:
len(nu_mcdf) / 1e6

In [ ]:
wfid = InFV(nu_mcdf.position, 50, 10, 10, 10).sum() / len(nu_mcdf)
wfid

In [ ]:
sint = (nu_mcdf.genie_mode != 0) & (nu_mcdf.genie_mode != 10) & (nu_mcdf.iscc==1)
wint = sint.sum() / len(nu_mcdf)
wint

In [ ]:
sint_E = (nu_mcdf.genie_mode != 0) & (nu_mcdf.genie_mode != 10) & (nu_mcdf.iscc==1) & (nu_mcdf.E > 1) & (nu_mcdf.E < 3.5)
wint = sint.sum() / len(nu_mcdf)
wint

In [ ]:
w_q2E = (sint & ((nu_mcdf.Q2 / nu_mcdf.E**2) < 0.2)).sum() / len(nu_mcdf)
w_q2E

In [ ]:
w_pke = (sint & (nu_mcdf.max_proton_ke <= 0.05)).sum() / len(nu_mcdf)
w_pke

In [ ]:
w_pke_pi = ((nu_mcdf.max_proton_ke <= 0.05) & (nu_mcdf.iscc + nu_mcdf.npi > 1)).sum() / len(nu_mcdf)
w_pke_pi

In [ ]:
1/(wfid*wint)

In [ ]:
1/(wfid*w_pke)

In [ ]:
1/(wfid*w_pke_pi)

In [ ]:
1/w_pke_pi

In [ ]:
nu_mcdf.scale

In [ ]:
1e8 / 35 / 100.